*This notebook is part of  course materials for CS 345: Machine Learning Foundations and Practice at Colorado State University.
Original versions were created by Asa Ben-Hur and updated by Ross Beveridge.
The content is availabe [on GitHub](https://github.com/asabenhur/CS345).*

*The text is released under the [CC BY-SA license](https://creativecommons.org/licenses/by-sa/4.0/), and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<a href="https://colab.research.google.com/github//asabenhur/CS345/blob/master/fall24/notebooks/module01_06_perceptron.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Predicting Valorant Match Outcomes Using Performance Metrics

By Hallie Gurr and Rose Ordway

In [10]:
import sqlite3
import pandas as pd

db_path = "data/valorant.sqlite"

conn = sqlite3.connect(db_path)

# tables = pd.read_sql_query(
#     "SELECT name FROM sqlite_master WHERE type='table';", conn
# )
# print(tables)
# df = pd.read_sql_query("SELECT * FROM matches LIMIT 10;", conn)
# print(df)

# df = pd.read_sql_query("SELECT * FROM matches;", conn)
# print("Matches columns:", df.columns.tolist())

# df_games = pd.read_sql_query("SELECT * FROM games;", conn)
# print("Games columns:", df_games.columns.tolist())

# df_game_rounds = pd.read_sql_query("SELECT * FROM game_rounds;", conn)
# print("Game Rounds columns:", df_game_rounds.columns.tolist())

# df_game_scores = pd.read_sql_query("SELECT * FROM Game_Scoreboard;", conn)
# print("Game Scores columns:", df_game_scores.columns.tolist())

#create csv file from tables
# df.to_csv("matches.csv", index=False)
# df_games.to_csv("games.csv", index=False)
# df_game_rounds.to_csv("game_rounds.csv", index=False)
# df_game_scores.to_csv("game_scores.csv", index=False)
import ast
roundhistory = pd.read_sql_query("Select RoundHistory from game_rounds LIMIT 1;", conn)
print(roundhistory.iloc[0]['RoundHistory'])

data = roundhistory.iloc[0]['RoundHistory']
data = ast.literal_eval(data)

print(type(data))
print(data[1])  

{1: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '1-0', 'WinType': 'elim', 'Team1Bank': 300.0, 'Team2Bank': 100.0, 'Team1BuyType': 'eco', 'Team2BuyType': 'eco'}, 2: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '2-0', 'WinType': 'elim', 'Team1Bank': 2700.0, 'Team2Bank': 8600.0, 'Team1BuyType': 'semi-buy', 'Team2BuyType': 'eco'}, 3: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '3-0', 'WinType': 'elim', 'Team1Bank': 12300.0, 'Team2Bank': 400.0, 'Team1BuyType': 'semi-buy', 'Team2BuyType': 'full-buy'}, 4: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '4-0', 'WinType': 'elim', 'Team1Bank': 23600.0, 'Team2Bank': 9800.0, 'Team1BuyType': 'full-buy', 'Team2BuyType': 'semi-eco'}, 5: {'RoundWinner': 'PHO ', 'ScoreAfterRound': '4-1', 'WinType': 'boom', 'Team1Bank': 24200.0, 'Team2Bank': 3300.0, 'Team1BuyType': 'full-buy', 'Team2BuyType': 'full-buy'}, 6: {'RoundWinner': 'PHO ', 'ScoreAfterRound': '4-2', 'WinType': 'boom', 'Team1Bank': 10200.0, 'Team2Bank': 10300.0, 'Team1BuyType': 'full-buy', 'Team2BuyType':

### A review of labeled data

**Definition.** A labeled dataset is a collection of feature vectors and their associated labels:
$$
\left( \;(\mathbf{x}_1, y_1),\; (\mathbf{x}_2, y_2),\ldots,(\mathbf{x}_N, y_N) \;\right)
$$
where $\mathbf{x}_i \in \mathbb{R}^d$ is a feature vector and $y_i$ is the label associated with $\mathbf{x}_i$.
(Recall that $\mathbb{R}^d$ is the set of all $d$-dimensional vectors with entries that are real numbers).  When dealing with classification problems, the labels belong to a set of discrete options, which is the set of classes that an object can belong to.

In this notebook we will focus on **binary classification** problems, i.e. problem with two classes of examples that we refer to as **positive and negative**.  Here we will use the labels $\{ -1, 1 \}$ to denote negative and positive examples, respectively.